In [ ]:
!pip3 install scikit-image --user

In [13]:
import os
import keras
import numpy as np
import random
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.models import model_from_json

from skimage import transform
from skimage.io import imread
from skimage.color import rgb2gray
from sklearn.metrics import classification_report, confusion_matrix

In [22]:
"""
    Preprocess image, normalizing and resizing it

    :param frame: RGBA frame
"""    
def preprocess_image(frame):
    
    # Normalize Pixel Values
    normalized_frame = frame/255.0 - 0.5
    
    # Resize
    preprocessed_frame = transform.resize(normalized_frame, IMAGE_PP_SIZE)
    
    return preprocessed_frame

"""
    Create 2D label list from 1D list
    
    :param labels: 1D label list
"""

def make_labels(labels):
    np_labels = np.zeros((len(labels), 16))
    
    for i in range(len(labels)):
        np_labels[i, labels[i]] = 1
    
    return np_labels

def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

def choices(l, k=1):
    new_list = []
    for i in range(k):
        new_list.append(random.choice(l))
    return new_list

## Define Globals

In [23]:
IMAGE_PP_SIZE = [150, 150]
EPOCHS = 1
BATCH_SIZE = 100
TRAIN_STEP = 1000
VAL_SIZE = 100

RELOAD_MODEL = True

## Load Model if it Exists, Otherwise Recreate it

In [19]:
if RELOAD_MODEL:
    try:
        json_file = open('model.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        model = model_from_json(loaded_model_json)
        # load weights into new model
        model.load_weights("model.h5")
    except FileNotFoundError:

        model = Sequential()
        model.add(Conv2D(20, kernel_size=(7, 7), strides=(1, 1),
                         activation='relu',
                         input_shape=(IMAGE_PP_SIZE[0], IMAGE_PP_SIZE[1], 1)))
        model.add(MaxPooling2D(pool_size=(4, 4), strides=(1, 1)))
        model.add(Conv2D(50, kernel_size = (5, 5), activation='relu'))
        model.add(MaxPooling2D(pool_size=(4, 4)))
        model.add(Flatten())
        model.add(Dense(1000, activation='relu'))
        model.add(Dense(1000, activation='relu'))
        model.add(Dense(16, activation='softmax'))
else:
    model = Sequential()
    model.add(Conv2D(20, kernel_size=(7, 7), strides=(1, 1),
                     activation='relu',
                     input_shape=(IMAGE_PP_SIZE[0], IMAGE_PP_SIZE[1], 1)))
    model.add(MaxPooling2D(pool_size=(4, 4), strides=(1, 1)))
    model.add(Conv2D(50, kernel_size = (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(4, 4)))
    model.add(Flatten())
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(16, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01),
              metrics=['accuracy'])

In [ ]:

#
# Define train, validation and test lists
#

train_labels = []
val_labels = []
test_labels = []

train_input = []
val_input = []
test_input = []

#
# Load validation dataset
#

folder = "/media/matheusmortatti/External/rvl-cdip"
labels_file = open(folder + "/labels/val.txt")

val_file = []
for line in labels_file:
    sp = line.split()
    val_file.append(sp)
    
#
# Open training list file and start training
#

labels_file = open(folder + "/labels/train.txt")
trained = 0
train_size = file_len(folder + "/labels/train.txt")

for i in range(EPOCHS):
    for line in labels_file:
        sp = line.split()
        pp_img = preprocess_image(imread(folder + "/images/" + sp[0]))

        train_input.append(pp_img)
        train_labels.append(int(sp[1]))

        if len(train_input) >= TRAIN_STEP:
            
            trained += len(train_input)

            #
            # Choose a subset of the validation data
            #

            ss_val = choices(val_file, k=VAL_SIZE)

            for v in ss_val:
                val_input.append(preprocess_image(imread(folder + "/images/" + v[0])))
                val_labels.append(int(v[1]))

            x_val = np.expand_dims(np.asarray(val_input), axis=3)
            y_val = make_labels(val_labels)

            #
            # Create training data
            #

            x_train = np.expand_dims(np.asarray(train_input), axis=3)
            y_train = make_labels(train_labels)

            model.fit(x_train, y_train,
                      batch_size=BATCH_SIZE,
                      epochs=1,
                      verbose=1,
                      validation_data=(x_val, y_val))

            #
            # Reset lists for next iteration
            #

            train_input = []
            train_labels = []
            val_input = []
            val_labels = []
            
            print("trained: " + str(trained) + " / " + str(train_size))


/home/matheusmortatti/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/matheusmortatti/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 5s 5ms/step - loss: 0.4828 - acc: 0.8640 - val_loss: 0.6091 - val_acc: 0.8000
trained: 1000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4994 - acc: 0.8510 - val_loss: 0.7580 - val_acc: 0.7900
trained: 2000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4621 - acc: 0.8670 - val_loss: 0.7548 - val_acc: 0.7600
trained: 3000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 3s 3ms/step - loss: 0.4665 - acc: 0.8550 - val_loss: 0.6681 - val_acc: 0.8300
trained: 4000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 3s 3ms/step - loss: 0.4081 - acc: 0.8730 - val_loss: 0.7587 - val_acc: 0.7600
trained:

1000/1000 [==============================] - 2s 2ms/step - loss: 0.3196 - acc: 0.8960 - val_loss: 0.5787 - val_acc: 0.8300
trained: 41000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3373 - acc: 0.8960 - val_loss: 0.3857 - val_acc: 0.8800
trained: 42000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3708 - acc: 0.8800 - val_loss: 0.6806 - val_acc: 0.7600
trained: 43000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3208 - acc: 0.9060 - val_loss: 0.8370 - val_acc: 0.7400
trained: 44000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3931 - acc: 0.8780 - val_loss: 0.4336 - val_acc: 0.8700
trained: 45000 / 320000
Train on 1000 samples, validate on 10

1000/1000 [==============================] - 2s 2ms/step - loss: 0.3210 - acc: 0.9050 - val_loss: 0.8635 - val_acc: 0.7900
trained: 81000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.2665 - acc: 0.9250 - val_loss: 0.5082 - val_acc: 0.8600
trained: 82000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.2523 - acc: 0.9250 - val_loss: 0.8044 - val_acc: 0.7800
trained: 83000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.2910 - acc: 0.9040 - val_loss: 0.8687 - val_acc: 0.8100
trained: 84000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.2901 - acc: 0.9070 - val_loss: 0.7456 - val_acc: 0.7600
trained: 85000 / 320000
Train on 1000 samples, validate on 10

1000/1000 [==============================] - 2s 2ms/step - loss: 0.1835 - acc: 0.9450 - val_loss: 0.4184 - val_acc: 0.9100
trained: 121000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.2423 - acc: 0.9280 - val_loss: 0.5407 - val_acc: 0.7900
trained: 122000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.2120 - acc: 0.9290 - val_loss: 0.6121 - val_acc: 0.8500
trained: 123000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.1977 - acc: 0.9410 - val_loss: 0.8227 - val_acc: 0.7800
trained: 124000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.1976 - acc: 0.9460 - val_loss: 0.8758 - val_acc: 0.7600
trained: 125000 / 320000
Train on 1000 samples, validate 

1000/1000 [==============================] - 2s 2ms/step - loss: 0.5216 - acc: 0.8360 - val_loss: 0.6243 - val_acc: 0.8700
trained: 161000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4793 - acc: 0.8640 - val_loss: 0.8144 - val_acc: 0.8200
trained: 162000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4428 - acc: 0.8690 - val_loss: 0.6045 - val_acc: 0.8000
trained: 163000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4758 - acc: 0.8510 - val_loss: 0.8402 - val_acc: 0.7900
trained: 164000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.5155 - acc: 0.8350 - val_loss: 0.6209 - val_acc: 0.8000
trained: 165000 / 320000
Train on 1000 samples, validate 

1000/1000 [==============================] - 2s 2ms/step - loss: 0.4960 - acc: 0.8510 - val_loss: 0.8867 - val_acc: 0.7300
trained: 201000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4365 - acc: 0.8760 - val_loss: 0.6935 - val_acc: 0.8000
trained: 202000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4547 - acc: 0.8640 - val_loss: 0.8861 - val_acc: 0.7700
trained: 203000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4440 - acc: 0.8520 - val_loss: 0.6630 - val_acc: 0.8100
trained: 204000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4956 - acc: 0.8590 - val_loss: 0.5162 - val_acc: 0.8100
trained: 205000 / 320000
Train on 1000 samples, validate 

1000/1000 [==============================] - 2s 2ms/step - loss: 0.4415 - acc: 0.8670 - val_loss: 0.7294 - val_acc: 0.7500
trained: 241000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4110 - acc: 0.8730 - val_loss: 0.6911 - val_acc: 0.8300
trained: 242000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4202 - acc: 0.8660 - val_loss: 0.5891 - val_acc: 0.7800
trained: 243000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4627 - acc: 0.8710 - val_loss: 0.7588 - val_acc: 0.8000
trained: 244000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3760 - acc: 0.8880 - val_loss: 0.7316 - val_acc: 0.7800
trained: 245000 / 320000
Train on 1000 samples, validate 

1000/1000 [==============================] - 2s 2ms/step - loss: 0.3431 - acc: 0.9010 - val_loss: 0.6032 - val_acc: 0.7700
trained: 281000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.5120 - acc: 0.8470 - val_loss: 0.8061 - val_acc: 0.8100
trained: 282000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4009 - acc: 0.8650 - val_loss: 0.4559 - val_acc: 0.8600
trained: 283000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4199 - acc: 0.8700 - val_loss: 0.7207 - val_acc: 0.7900
trained: 284000 / 320000
Train on 1000 samples, validate on 100 samples
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4134 - acc: 0.8760 - val_loss: 0.6894 - val_acc: 0.8100
trained: 285000 / 320000
Train on 1000 samples, validate 

## Save Model and Weights

In [16]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

In [ ]:
np_img_type = np.zeros((len(img_type), dt))

print(len(img_type))

for i in range(len(img_type)):
    np_img_type[i, img_type[i]] = 1
print(np_img_type)

In [ ]:
x_train = np.expand_dims(np.asarray(imgs[:2000]), axis=3)
y_train = np_img_type[:2000,:]

x_val = np.expand_dims(np.asarray(imgs[2000:4000]), axis=3)
y_val = np_img_type[2000:4000,:]

x_test = np.expand_dims(np.asarray(imgs[4000:]), axis=3)
y_test = np_img_type[4000:,:]

print(y_train.shape)

In [ ]:
model.fit(x_train, y_train,
          batch_size=100,
          epochs=2,
          verbose=1,
          validation_data=(x_val, y_val))

In [ ]:
#Confution Matrix and Classification Report
Y_pred = model.predict_classes(x_test)

In [ ]:
print(Y_pred)

In [ ]:
y_pred = Y_pred
y_true = np.argmax(y_test, axis=1)
print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))